In [0]:
!pip install q keras==2.3.0
!pip install tensorflow_hub

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt

%tensorflow_version 2.1.0
import tensorflow as tf

import tensorflow_hub as hub

from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.utils import to_categorical

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers import Input
from keras.models import Model

from sklearn.metrics import classification_report

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import pickle
import cv2
import os
import h5py
import keras
print(keras.__version__)
print(tf.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.1.0`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


Using TensorFlow backend.


2.3.0
2.2.0-rc3


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def plot_training(H, N, plotPath):
	# construct a plot that plots and saves the training history
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
	plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
	plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
	plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
	plt.title("Training Loss and Accuracy")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend(loc="lower left")
	plt.savefig(plotPath)

In [0]:
data_root='/content/drive/My Drive/Audio_Quality_Analysis/data/dataset_1/train'
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)

datagen_kwargs = dict(rescale=1./255, validation_split=.20)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory( TRAINING_DATA_DIR, subset="validation",
                                                      shuffle=True, target_size=IMAGE_SHAPE)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(TRAINING_DATA_DIR, subset="training",
                                                      shuffle=True, target_size=IMAGE_SHAPE)

Found 2240 images belonging to 2 classes.
Found 8963 images belonging to 2 classes.


In [0]:
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
print(dataset_labels)
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

[('good', 0), ('noise', 1)]
['Good' 'Noise']


In [0]:
le = LabelEncoder()
labels = le.fit_transform(dataset_labels)
#labels = np_utils.to_categorical(labels, 3)
print(labels)

with open('/content/drive/My Drive/Audio_Quality_Analysis/model/labelEncoderFile_VGG16_TL_FineTuning_04262020_v1.pkl', 'wb') as fid:
    pickle.dump(le, fid)

[0 1]


In [0]:
baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
	layer.trainable = False

model.summary()

print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

58892288/58889256 [==============================] - 1s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [0]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

hist = model.fit_generator( train_generator, epochs=14, verbose=1,
                            steps_per_epoch=steps_per_epoch,
                            validation_data=valid_generator,
                            validation_steps=val_steps_per_epoch
                           ).history

model.save("/content/drive/My Drive/Audio_Quality_Analysis/model/AudioQuality_VGG16_TL_GD_FineTuning_V1_04227.h5")
model.save_weights("/content/drive/My Drive/Audio_Quality_Analysis/model/AudioQualityWeights_VGG16_TL_GD_FineTuning_V1_0427.h5")

plot_training(hist, 20, 'warmup.png')

In [0]:
for layer in baseModel.layers[15:]:
	layer.trainable = True

# loop over the layers in the model and show which ones are trainable
# or not
for layer in baseModel.layers:
	print("{}: {}".format(layer, layer.trainable))

In [0]:
print("[INFO] re-compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [0]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

hist = model.fit_generator( train_generator, epochs=10, verbose=1,
                            steps_per_epoch=steps_per_epoch,
                            validation_data=valid_generator,
                            validation_steps=val_steps_per_epoch
                           ).history

model.save("/content/drive/My Drive/Audio_Quality_Analysis/model/AudioQuality_VGG16_TL_GD_FineTuning_V1_04228.h5")
model.save_weights("/content/drive/My Drive/Audio_Quality_Analysis/model/AudioQualityWeights_VGG16_TL_GD_FineTuning_V1_0428.h5")

plot_training(hist, 10, 'warmup.png')

In [0]:
from keras import models
model = tf.keras.models.load_model('/content/drive/My Drive/Audio_Quality_Analysis/model/AudioQuality_VGG16_TL_GD_FineTuning_V1_04227.h5')
model.load_weights('/content/drive/My Drive/Audio_Quality_Analysis/model/AudioQualityWeights_VGG16_TL_GD_FineTuning_V1_0427.h5')
model.summary()